#### Imports

In [1]:
import PyPDF2
import csv
import re
import json
import os

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [83]:
pdfFileObj = open('2007/directory_2007.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 680


In [87]:
def clean_key_page(raw_abbr_key):
    raw_abbr_key = raw_abbr_key.replace('\n', ' ').replace('(', ' ').replace(')', ' ').split(' ')
    raw_abbr_key[:] = [l for l in raw_abbr_key if (l.strip()!= '')]
    i = 0
    while i < len(raw_abbr_key):
        #print(raw_abbr_key[i])
        if raw_abbr_key[i].isupper():
            abbr_key.append(raw_abbr_key[i])
        i += 1

In [88]:
abbr_key = []

In [89]:
raw_abbr_key1 = pdfReader.getPage(4).extractText()
clean_key_page(raw_abbr_key1)
raw_abbr_key2 = pdfReader.getPage(5).extractText()
clean_key_page(raw_abbr_key2)

In [90]:
abbr_key

['TO',
 'FACILITY',
 'CODES*',
 'SA',
 'MH',
 'MH-SA',
 'GH',
 'TX',
 'DT',
 'MM',
 'DM',
 'BU',
 'HH',
 'HI',
 'OP',
 'PH',
 'RS',
 'RL',
 'AD',
 'CO',
 'HV',
 'HIV/AIDS',
 'GL',
 'SE',
 'PW',
 'WN',
 'MN',
 'BC',
 'DU',
 'DUI/DWI',
 'CJ',
 'SF',
 'MD',
 'MC',
 'SI',
 'PI',
 'MI',
 'VA,',
 'TRICARE',
 'AR',
 'SS',
 'PA',
 'AH',
 'ASL',
 'SP',
 'TO',
 'SPECIAL',
 'LANGUAGE',
 'SERVICES',
 'NOTE:',
 'N2',
 'N15',
 'N24',
 'N33',
 'N3',
 'N18',
 'N25',
 'N34',
 'N5',
 'N19',
 'N26',
 'N35',
 'N6',
 'N20',
 'N28',
 'N37',
 'N8',
 'N21',
 'N29',
 'N40',
 'N13',
 'N22',
 'N30',
 'N41',
 'N14',
 'N23',
 'N32',
 'F1',
 'F28',
 'F52',
 'F74',
 'F2',
 'F30',
 'F54',
 'F75',
 'F3',
 'F31',
 'F55',
 'F77',
 'F4',
 'F33',
 'F56',
 'SP',
 'F5',
 'F34',
 'F57',
 'F78',
 'F8',
 'F35',
 'F58',
 'F79',
 'F9',
 'F36',
 'F59',
 'F81',
 'F10',
 'F37',
 'F60',
 'F83',
 'F11',
 'F38',
 'F61',
 'F86',
 'F15',
 'F39',
 'F62',
 'F87',
 'F16',
 'F40',
 'F63',
 'F88',
 'F17',
 'F41',
 'F64',
 'F89',
 'F18',
 'F4

- Import the key, don't need to run everything above again.

In [3]:
with open("abbr_key.json") as abbr_key_json:
    abbr_key = json.load(abbr_key_json)

In [4]:
abbr_key

{'GHF': 'General Health Services',
 'MHF': 'Mental Health Treatment Services',
 'MHSAF': 'Mix of Mental Health and Substance Abuse',
 'SAF': 'Substance Abuse Treatment Services',
 'ACM': 'Acamprosate (Campral®)',
 'BM': 'Buprenorphine, maintenance',
 'BMW': 'Buprenorphine, maintenance, for, a, pre-',
 'BU': 'Buprenorphine, used, in, treatment',
 'DB': 'Buprenorphine, detoxi˝cation',
 'DM': 'Methadone, detoxi˝cation',
 'DSF': 'Disul˝ram (Antabuse®)',
 'DT': 'Detoxi˝cation',
 'HH': 'Transitional, housing, or, halfway, house',
 'METH': 'Methadone',
 'MM': 'Methadone maintenance',
 'MMW': 'Methadone maintenance for pre-determined',
 'MOA': 'Accepts clients on opioid medication',
 'MPD': 'Medications for psychiatric disorders',
 'NMOA': 'Does not use medication for opioid addiction',
 'NOOP': 'Does not treat opioid addiction',
 'NXN': 'Naltrexone, (oral)',
 'OTP': 'SAMHSA-certi˝ed Opioid Treatment Program',
 'OTPA': 'All clients in Opioid Treatment Program',
 'PAIN': 'Use methadone/buprenor

#### Scraper test on a page of data
- data starts on page 15 (getPage(14))

In [16]:
'CMHA' in abbr_key

False

In [7]:
def clean_data(rawdata):
    '''
    Roughly split data, returns list of list.
    Inputs:
        rawdata_from_page(list): list of items on the page that were seperated by /n
    '''
    #split erroneously connected data
    data = []
    temp = []
    i = 0
    while i < len(rawdata) - 1:
        rawdata[i] = rawdata[i].replace('˜', '').replace('  ', ' ').strip()
        #if i is the last line of keys, i+1 is facility name, i-1 is the phone number or other line of keys
        if rawdata[i].isupper() and (rawdata[i].split()[0] in abbr_key) and \
        (not rawdata[i+1].split()[0] in abbr_key) and (rawdata[i-1][0] == '(' or rawdata[i-1].isupper()): 
            print(rawdata[i])
            print(' ')
            temp.append(rawdata[i])
            data.append(temp)
            temp = []
        else:
            temp.append(rawdata[i])
        i += 1
    temp.append(rawdata[i])
    data.append(temp)
    return data


def clean_facility_name(fname):
    '''
    Split facility name into ["junk", actual name]
    fname(str): name of the facility
    '''
    front = ''
    i = 0
    while i < len(fname) - 1:
        if (fname[i].isupper() or fname[i] == ' ')\
        and (fname[i+1].isupper() or fname[i+1] == ' '):
            front += fname[i]
            i += 1
        else:
            break
            
    if front in abbr_key:
        return [front, fname[i:]]
    
    #if its just capitalized name...
    elif front == 'YMCA' or front == 'BAART' or fname[i:i+3] == 'Inc':
        return ['', fname]
    #if it is accidentally included city name
    else:
        return ['', fname[i:]]
    
    
def clean_all_facility_name(data):
    '''
    Move and link wrongly splitted data, changes in place
    '''
    i = 0
    while i < len(data):
        facility_name_raw = data[i][0]
        if facility_name_raw[:2].isupper():
            code, fac_name = clean_facility_name(facility_name_raw)
            data[i][0] = fac_name
            if i != 0:
                data[i - 1].append(code)
        i += 1

In [8]:
def split_center_street(cn_sa, page):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list
    '''
    #Exception to split at number :
        #When it ends with "P.O. Box",
        #When the number being found is at the very front (wrongly included page number or 
        #center name starts with number)
        
    #If the number is at the end(suite xxx or building xxx) lets include the word in front of it)
    #print(cn_sa)
    #print(page)
    if cn_sa[0].isnumeric():
        temp = cn_sa.split()
        if temp[0] == str(page):
            cn_sa = ' '.join(temp[1:])
            
    m = re.finditer(r"\d+", cn_sa)

    for num in m:
        span = num.span()
        if span[0] != 0:    
            if cn_sa[span[0] - 9:span[0] - 1] == 'P.O. Box':
                return ([cn_sa[:span[0] - 10], cn_sa[span[0] - 9:]])
            if span[1] != len(cn_sa):
                return ([cn_sa[:span[0] - 1], cn_sa[span[0]:]])
            if span[1] == len(cn_sa):
                return ([' '.join(cn_sa.split()[:-2]), ' '.join(cn_sa.split()[-2:])])
    return [cn_sa, '']

In [9]:
def split_dc(year, page, dc, writer):
    '''
    Writes data in dc, returns appending data if any
    '''
    i = 0
    data_holder = [year, page]
    #Center name, street address
    cn_sa = ''
    while (',' not in dc[i]) or not dc[i][-5:].isnumeric(): #(len(dc[i]) < 2) or 
        cn_sa += ' ' + dc[i]
        i += 1
    data_holder += split_center_street(cn_sa.strip(), page)

    #City, State, Postal_code
    city, state_postcode = dc[i].split(',')
    state, postcode = state_postcode.split()
    data_holder += [city, state, postcode]
    i += 1
    #phone + keys
    phone = dc[i]
    i += 1
    while not dc[i][0:2].isupper():
        phone += ' ' + dc[i]
        i += 1
    if phone[-1].isupper():
        key_start = re.search(r'[A-Z]{2}', phone).start()
        data_holder.append(phone[:key_start])
        data_holder.append(phone[key_start:] + ' ' + ' '.join(dc[i:]))
    else:
        key_start = len(phone)
        data_holder.append(phone)
        data_holder.append(' '.join(dc[i:]))
        
    if not data_holder[-1].isupper(): #if the last line of keys also contains another center
        keys_tail = data_holder[-1]
        tail_start = re.search(r'[A-Z][a-z]', keys_tail).start()
        data_holder[-1] = keys_tail[: tail_start]
        return keys_tail[tail_start:]
        
    #write data
    writer.writerow(data_holder)
    
    

def split_dc_tail(year, page, dc, writer):
    city_pos = re.search(',', dc).start()
    before = dc[:city_pos]
    state = dc[city_pos+2: city_pos+4]
    postal_code = dc[city_pos+5:city_pos+10]
    after = dc[city_pos + 11:]

    before = before.split(' ')
    city = before.pop()
    cn = ''
    addr = ''
    i = 0
    while i < len(before):
        if before[i].isnumeric():
            break
        else:
            cn += ' ' + before[i]
        i += 1
    addr = ' '.join(before[i:])

    keys_pos = re.search(r'[A-Z]', after).start()
    phone = after[:keys_pos]
    keys = after[keys_pos:]
    writer.writerow([year, page, cn.strip(), addr, city, state, postal_code, phone, keys])
    #print(cn.strip(), addr, city, state, postal_code, phone, keys)

In [10]:
def write_data(data, year, page, filename):
    '''
    Write data into right columns
    '''
    with open(filename, 'a', newline = '') as file:
        writer = csv.writer(file)
        #loop over centers
        tails = []
        for dc in data:
            tail = split_dc(year, page, dc, writer)
            if tail != None:
                #print(tail)
                tails.append(tail)
        for tail in tails:
            split_dc_tail(year, page, tail, writer)

## Test on a bunch of pages

In [11]:
def run(year, start_page):
    
    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    end_page = pdfReader.numPages - 1
    
    #open/create csv
    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])

    for i in range(start_page, end_page + 1):
        page = pdfReader.getPage(i).extractText()
        rawdata = page.split('\n')
        #delete na lines
        rawdata[:] = [l.strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]
        #clean
        if rawdata and not (rawdata == [str(i) + 'KEY']):
            data = clean_data(rawdata)
            clean_all_facility_name(data)
            write_data(data, year, i + 1, save_as_filename)

In [13]:
page = pdfReader.getPage(14).extractText()
rawdata = page.split('\n')
rawdata[:] = [l.strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]
rawdata

['ABBEVILLESpectraCare',
 'Henry County/OP',
 '219 Dothan Road',
 'Abbeville, AL 36310',
 '(334) 673-2143',
 'MHSAF',
 'SA ACM MPD NXN VTRL',
 'MOA',
 'CBT SACA TRC REBT TWFA',
 'MOTI ANG MXM RELP',
 'SMPD',
 'OP',
 'ORT',
 'PVT',
 'STAG STMH',
 'SF MD',
 'MC SI PI MI FSA',
 'SS',
 'CO GL VET CJ',
 'SE PW WN MN HV TRMA XA DV',
 'TAY',
 'CM HEOH SSA SMHD CSAA',
 'CMHA ISC BABA DAUT TBS DP ACC',
 'SSD AOSS HAEC TAEC SAE MHS ICO',
 'GCO MCO',
 'CHLD YAD ADLT',
 'FEM',
 'MALE',
 'AH',
 'Shelby County Treatment Center',
 '750 Highway 31 South',
 'Alabaster, AL 35007',
 '(205) 216-0200',
 'SAF',
 'SA MM MMW BUM BMW RPN',
 'BU DSF ACM MPD NXN VTRL METH',
 'OTPA OTP',
 'CBT DBT SACA TRC',
 'REBT TWFA BIA CMI MOTI ANG',
 'MXM RELP',
 'SMPD',
 'OP OMB OIT',
 'ORT',
 'PVT',
 'STAG STMH CARF',
 'SF',
 'MD PI',
 'SS',
 'CO GL VET ADM MF CJ',
 'SE PW WN MN HV TRMA XA DV TAY',
 'CM PEER HEOH HS NRT NSC STU',
 'TCC SSA SMHD CSAA CMHA OPC',
 'ISC BABA DAUT SHB SHC HIVT STDT',
 'TBS DP ACC SSD AOSS EMP 

In [14]:
data = clean_data(rawdata)
data

CM HEOH SSA SMHD CSAA
 
AH
 
AH SP
 
AH
 
MALE
 
FEM MALE
 
FEM MALE
 
MALE
 
FEM MALE
 


[['ABBEVILLESpectraCare',
  'Henry County/OP',
  '219 Dothan Road',
  'Abbeville, AL 36310',
  '(334) 673-2143',
  'MHSAF',
  'SA ACM MPD NXN VTRL',
  'MOA',
  'CBT SACA TRC REBT TWFA',
  'MOTI ANG MXM RELP',
  'SMPD',
  'OP',
  'ORT',
  'PVT',
  'STAG STMH',
  'SF MD',
  'MC SI PI MI FSA',
  'SS',
  'CO GL VET CJ',
  'SE PW WN MN HV TRMA XA DV',
  'TAY',
  'CM HEOH SSA SMHD CSAA'],
 ['CMHA ISC BABA DAUT TBS DP ACC',
  'SSD AOSS HAEC TAEC SAE MHS ICO',
  'GCO MCO',
  'CHLD YAD ADLT',
  'FEM',
  'MALE',
  'AH'],
 ['Shelby County Treatment Center',
  '750 Highway 31 South',
  'Alabaster, AL 35007',
  '(205) 216-0200',
  'SAF',
  'SA MM MMW BUM BMW RPN',
  'BU DSF ACM MPD NXN VTRL METH',
  'OTPA OTP',
  'CBT DBT SACA TRC',
  'REBT TWFA BIA CMI MOTI ANG',
  'MXM RELP',
  'SMPD',
  'OP OMB OIT',
  'ORT',
  'PVT',
  'STAG STMH CARF',
  'SF',
  'MD PI',
  'SS',
  'CO GL VET ADM MF CJ',
  'SE PW WN MN HV TRMA XA DV TAY',
  'CM PEER HEOH HS NRT NSC STU',
  'TCC SSA SMHD CSAA CMHA OPC',
  'ISC B

In [51]:
page = pdfReader.getPage(14).extractText()
rawdata = page.split('\n')
rawdata

['',
 'KEY',
 'ALABAMAALEXANDER CITYLighthouse of Tallapoosa County Inc',
 'Substance Abuse Rehab Program/Resid',
 '36 Franklin Street',
 'Alexander City, AL 35010',
 '(256) 234-4894SA ˜ TX HH ',
 '˜ RL ',
 '˜ MN ',
 '˜ SF',
 'Anniston Fellowship House Inc',
 '106 East 22nd Street',
 'Anniston, AL 36201',
 '(256) 236-7229SA ˜ TX HH ',
 '˜ RL ',
 '˜ MN ',
 '˜ SFBridge Inc',
 'Recovery Center for Teens/Ashville',
 '474 10th Street',
 'Ashville, AL 35953',
 '(205) 594-3111SA ˜ TX ',
 '˜ OP ',
 '˜ AD CO CJ ',
 '˜ SF MD SI PI ',
 'MI ',
 '˜ SS PA ',
 '˜ AHATTALLA',
 'Rapha Christian Home',
 '677 West Covington Avenue',
 'Attalla, AL 35954',
 '(256) 538-7458SA ˜ TX HH ',
 '˜ RL ',
 '˜ SFNorthwest Alabama Treatment Center',
 '4204 Edmonton Drive',
 'Bessemer, AL 35022',
 '(205) 425-1200Methadone Clients Only',
 'SA ˜ TX MM ',
 '˜ OP ',
 '˜ PW ˜ SF ',
 '˜ AHAlcohol and Drug Abuse',
 'Treatment Centers Inc',
 '2701 Je˜erson Avenue SW',
 'Birmingham, AL 35211',
 '(205) 923-6552Intakes: (205) 923

In [55]:
data = clean_data(rawdata)

SF
 
MN
 
SS PA
 
RL
 
PW SF
 
HV SF MD
 
SF MD
 
MN
 
SS PA
 
OP
 
SF SI PI MI
 
SF PI MI
 
SF MD MC SI PI
 
SF MD
 
SF MD
 
OP
 
SS PA
 


In [56]:
data

[['',
  'KEY',
  'ALABAMAALEXANDER CITYLighthouse of Tallapoosa County Inc',
  'Substance Abuse Rehab Program/Resid',
  '36 Franklin Street',
  'Alexander City, AL 35010',
  '(256) 234-4894SA TX HH',
  'RL',
  'MN',
  'SF'],
 ['Anniston Fellowship House Inc',
  '106 East 22nd Street',
  'Anniston, AL 36201',
  '(256) 236-7229SA TX HH',
  'RL',
  'MN'],
 ['SFBridge Inc',
  'Recovery Center for Teens/Ashville',
  '474 10th Street',
  'Ashville, AL 35953',
  '(205) 594-3111SA TX',
  'OP',
  'AD CO CJ',
  'SF MD SI PI',
  'MI',
  'SS PA'],
 ['AHATTALLA',
  'Rapha Christian Home',
  '677 West Covington Avenue',
  'Attalla, AL 35954',
  '(256) 538-7458SA TX HH',
  'RL'],
 ['SFNorthwest Alabama Treatment Center',
  '4204 Edmonton Drive',
  'Bessemer, AL 35022',
  '(205) 425-1200Methadone Clients Only',
  'SA TX MM',
  'OP',
  'PW SF'],
 ['AHAlcohol and Drug Abuse',
  'Treatment Centers Inc',
  '2701 Jeerson Avenue SW',
  'Birmingham, AL 35211',
  '(205) 923-6552Intakes: (205) 923-6552x10 (205

In [33]:
year = 2007
start_page = 14

dir_filename = '{0}/directory_{0}.pdf'.format(year)
pdfFileObj = open(dir_filename, 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

In [34]:
page = pdfReader.getPage(start_page).extractText()
rawdata = page.split('\n')

In [35]:
rawdata

['',
 'KEY',
 'ALABAMAALEXANDER CITYLighthouse of Tallapoosa County Inc',
 'Substance Abuse Rehab Program/Resid',
 '36 Franklin Street',
 'Alexander City, AL 35010',
 '(256) 234-4894SA ˜ TX HH ',
 '˜ RL ',
 '˜ MN ',
 '˜ SF',
 'Anniston Fellowship House Inc',
 '106 East 22nd Street',
 'Anniston, AL 36201',
 '(256) 236-7229SA ˜ TX HH ',
 '˜ RL ',
 '˜ MN ',
 '˜ SFBridge Inc',
 'Recovery Center for Teens/Ashville',
 '474 10th Street',
 'Ashville, AL 35953',
 '(205) 594-3111SA ˜ TX ',
 '˜ OP ',
 '˜ AD CO CJ ',
 '˜ SF MD SI PI ',
 'MI ',
 '˜ SS PA ',
 '˜ AHATTALLA',
 'Rapha Christian Home',
 '677 West Covington Avenue',
 'Attalla, AL 35954',
 '(256) 538-7458SA ˜ TX HH ',
 '˜ RL ',
 '˜ SFNorthwest Alabama Treatment Center',
 '4204 Edmonton Drive',
 'Bessemer, AL 35022',
 '(205) 425-1200Methadone Clients Only',
 'SA ˜ TX MM ',
 '˜ OP ',
 '˜ PW ˜ SF ',
 '˜ AHAlcohol and Drug Abuse',
 'Treatment Centers Inc',
 '2701 Je˜erson Avenue SW',
 'Birmingham, AL 35211',
 '(205) 923-6552Intakes: (205) 923

In [18]:
clean_data(rawdata)

[['',
  'KEY',
  'ALABAMAALEXANDER CITYLighthouse of Tallapoosa County Inc',
  'Substance Abuse Rehab Program/Resid',
  '36 Franklin Street',
  'Alexander City, AL 35010',
  '(256) 234-4894SA  TX HH',
  'RL',
  'MN',
  'SF'],
 ['Anniston Fellowship House Inc',
  '106 East 22nd Street',
  'Anniston, AL 36201',
  '(256) 236-7229SA  TX HH',
  'RL',
  'MN'],
 ['SFBridge Inc',
  'Recovery Center for Teens/Ashville',
  '474 10th Street',
  'Ashville, AL 35953',
  '(205) 594-3111SA  TX',
  'OP',
  'AD CO CJ',
  'SF MD SI PI',
  'MI',
  'SS PA',
  'AHATTALLA',
  'Rapha Christian Home',
  '677 West Covington Avenue',
  'Attalla, AL 35954',
  '(256) 538-7458SA  TX HH',
  'RL'],
 ['SFNorthwest Alabama Treatment Center',
  '4204 Edmonton Drive',
  'Bessemer, AL 35022',
  '(205) 425-1200Methadone Clients Only',
  'SA  TX MM',
  'OP',
  'PW  SF'],
 ['AHAlcohol and Drug Abuse',
  'Treatment Centers Inc',
  '2701 Jeerson Avenue SW',
  'Birmingham, AL 35211',
  '(205) 923-6552Intakes: (205) 923-6552x10

In [391]:
#debug purposes
year = 2012
pagen = 424
filename = str(year)+'.csv'

pdfFileObj = open('directory_{}.pdf'.format(str(year)), 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", "State", "Postal_code", "Phone", "Keys"])

page = pdfReader.getPage(pagen).extractText()
rawdata = page.split('\n')
#delete na lines
rawdata[:] = [l.strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]
if rawdata and not (rawdata == [str(pagen) + 'KEY']):
    data = clean_data(rawdata)
    clean_all_facility_name(data)
    write_data(data, year, pagen, filename)

True

In [362]:
s = 'SAFSA DT ACM MPD NXN VTRL NMOACBT DBT SACA TRC REBT TWFA BIA CMI MOTI ANG RELP SMONRES OP RS RL RD OD ODT OITPVTSTAG STDH JCSF PI MI CO GL VET ADM MF CJ SE WN MN HV TRMA XA DV TAYCM PEER HS NRT NSC STU TCC SSA SMHD CSAA CMHA BABA DAUT SHB SHC HIVT STDT TBS DP ACC SSD SAE TA MHS ACU SHG ADTX BDTX CDTX MDTX ODTX ICO GCO FCO MCOYAD ADLTFEMMALE'
s.replace('', ' ')

'SAF SA DT ACM MPD NXN VTRL NMOA CBT DBT SACA TRC REBT TWFA BIA CMI MOTI ANG RELP  SMON RES OP RS RL RD OD ODT OIT PVT STAG STDH JC SF PI MI  CO GL VET ADM MF CJ SE WN MN HV TRMA XA DV TAY CM PEER HS NRT NSC STU TCC SSA SMHD CSAA CMHA BABA DAUT SHB SHC HIVT STDT TBS DP ACC SSD SAE TA MHS ACU SHG ADTX BDTX CDTX MDTX ODTX ICO GCO FCO MCO YAD ADLT FEMMALE'